# LIMPIEZA TABLA CLIENTES Y CARGA A SILVER

In [0]:
from pyspark.sql.functions import col, when, regexp_replace, length, current_timestamp

# 1. Cargar datos desde Bronze
# Nota: Usamos los nombres de columna ya sanitizados por el proceso de ingesta previo
df_bronze = spark.read.table("workspace.bronze.clientes")

# 2. Aplicar transformaciones de limpieza y normalización
df_silver = df_bronze.select(
    col("tipo_identificacion").cast("string"),
    col("numero_identificacion").cast("string"),
    col("razon_social").cast("string"),
    # Lógica: Si Nombre Comercial es nulo/vacío, usar Razon Social
    when(
        (col("nombre_comercial").isNull()) | (col("nombre_comercial") == "") | (col("nombre_comercial") == "nan"), 
        col("razon_social")
    ).otherwise(col("nombre_comercial")).alias("nombre_comercial"),
    col("direccion").cast("string"),
    # Limpieza de Teléfono: Solo números, nulo si tiene menos de 7 dígitos
    when(
        length(regexp_replace(col("telefono_1"), r'[^0-9]', '')) < 7, 
        None
    ).otherwise(regexp_replace(col("telefono_1"), r'[^0-9]', '')).alias("telefono"),
    col("email").cast("string"),
    col("grupo").cast("string")
).filter(
    # Filtrar registros sin identificación
    (col("numero_identificacion").isNotNull()) & 
    (col("numero_identificacion") != "") & 
    (col("numero_identificacion") != "nan")
).withColumn("fecha_carga_silver", current_timestamp()) # Campo de auditoría

# 3. Guardar en esquema Silver
spark.sql("CREATE SCHEMA IF NOT EXISTS workspace.silver")
df_silver.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("workspace.silver.clientes")

print("✅ Tabla 'clientes' procesada exitosamente en Capa Silver.")